In [1]:
# using the CM metadata, create an Elasticsearch index with pages' metadata
import os, pprint, sys, time, csv, json
root = '/data/cm/data/'
root_out = '/data/cm/output/'

In [2]:
metadata = list(csv.reader(open(root+'metadata/CM_Seiten_Metadaten.csv', 'r'), delimiter='\t'))

In [3]:
j_meta = json.load(open('/home/rovera/cm/scripts/metadata/j_metadata.json', 'r', encoding='utf-8'))

In [4]:
for k,v in j_meta.items():
    print(k,v)
    break

2710055 {'dateIssued': '1919', 'place': 'München', 'ident': '2296512-9', 'lang': 'ger', 'subtitle': 'Zeitschr. der Jugend', 'title': 'Der neue Anfang', 'publisher': 'Verl. Der Neue Anfang'}


In [5]:
print(metadata[0])
metadata = {line[0]: line for line in metadata[1:]}

['VLID_Seite', 'OT_PATH', 'VLID_Zs', 'VLID_Parent', 'Parent Knotentyp', 'Parent-Type', 'Zs_Caption', 'Volume_Caption', 'Heft_Caption', 'Aufsatz_Caption', 'Datum', 'Seite (OT_SORT)', 'Seite_Caption', 'Seitenzahl_Caption']


In [6]:
# check overlap between VLIDs in my text repository and VLIDs in metadata

overlap = {'is_there': 0, 'is_not_there': 0}
for r, dirs, files in os.walk(root_out+'text/'):
    for file in files:
        try:
            vlid = file.split('.')[0].split('_')[1]
            if vlid in metadata:
                overlap['is_there'] +=1
            else:
                overlap['is_not_there'] += 1
        except IndexError:
            print(r, file)

In [6]:
for journal in os.listdir(root_out+'text/'):
    
    start = time.time()
    print('Processing', journal+'...', end=" ")
    
    # create a new folder for output (don't create the whole hierarchy, only journal folder)
    if not os.path.exists(root_out+"es_meta_fra/"+journal):
        os.makedirs(root_out+"es_meta_fra/"+journal)
        
    for r, dirs, files in os.walk(root_out+'text/'+journal+'/'):
        
        for file in files:
            filename = file.split('.')[0]
            vlid = file.split('.')[0].split('_')[1]
            
            if vlid in metadata:
            
                text = open(r+'/'+file).read()
            
                try:
                    j_title = metadata[vlid][6]
                except Error:
                    j_title = ""
                    
                try:
                    lang = j_meta[journal]['lang']
                except KeyError:
                    lang = ""
                    
                try:
                    publisher = j_meta[journal]['publisher']
                except KeyError:
                    publisher = ""
                    
                try:
                    volume = metadata[vlid][7]
                except Error:
                    volume = ""
                
                try:
                    heft = metadata[vlid][8]
                except Error:
                    heft = ""
                    
                try:
                    aufsatz = metadata[vlid][9]
                except Error:
                    aufsatz = ""
                    
                try:
                    date = metadata[vlid][10]
                except Error:
                    date = ""
                    
                try:
                    place = j_meta[journal]['place']
                except KeyError:
                    place = ""
                    
                try:
                    if metadata[vlid][13] != "(null)":
                        page = metadata[vlid][13]
                    else:
                        page = ""
                except Error:
                    page = ""
                    
                try:
                    vlid_journal = metadata[vlid][2]
                except Error:
                    vlid_journal = ""
                    
                try:
                    vlid_page = metadata[vlid][0]
                except Error:
                    vlid_page = ""
            
                es_header = {'_id': filename, '_index': 'cm_meta'}
                es_body = {'text': text,
                'dateIssued': date,
                'lang': lang,
                'place': place,
                'j_title': j_title,
                'publisher': publisher,
                'volume': volume,
                'heft': heft,
                'aufsatz': aufsatz,
                'page': page,
                'vlid_journal': vlid_journal,
                'vlid_page': vlid_page
                                    }
            
                es = []
                es.append(json.dumps(es_header, ensure_ascii=False))
                es.append(json.dumps(es_body, ensure_ascii=False))
                with open(root_out+"es_meta_fra/"+journal+"/"+filename+".json", "w", encoding="utf-8") as write_file:
                    write_file.write("\n".join(es))
    end = time.time()
    task_duration = round(end-start, 2)
    print('done in', str(task_duration))

Processing 2710055... done in 0.23
Processing 9616701... done in 3.66
Processing 2580773... done in 4.72
Processing 2895450... done in 13.56
Processing 2727810... done in 10.77
Processing 3129675... done in 4.35
Processing 6492429... done in 2.16
Processing 2438141... done in 20.18
Processing 2827798... done in 2.0
Processing 5438908... done in 0.54
Processing 4782723... done in 5.18
Processing 7506414... done in 1.9
Processing 9054153... done in 0.12
Processing 9366335... done in 0.02
Processing 4889892... done in 1.01
Processing 2425321... done in 2.1
Processing 6492329... done in 0.31
Processing 700673... done in 0.04
Processing 4895413... done in 0.73
Processing 2542917... done in 9.17
Processing 9583346... done in 0.71
Processing 2549643... done in 0.59
Processing 10638461... done in 0.99
Processing 3139318... done in 0.94
Processing 9620169... done in 0.86
Processing 9051331... done in 0.01
Processing 4781937... done in 0.47
Processing 4911569... done in 0.11
Processing 2625941..

Processing 4891256... done in 1.86
Processing 2417805... done in 0.93
Processing 9569929... done in 0.13
Processing 7242850... done in 1.91
Processing 4864325... done in 3.72
Processing 537862... done in 0.15
Processing 9626648... done in 0.04
Processing 9230935... done in 0.23
Processing 6492385... done in 1.25
Processing 8553624... done in 10.47
Processing 2316602... done in 13.0
Processing 2692297... done in 15.7
Processing 2709933... done in 0.48
Processing 2829132... done in 34.77
Processing 3062972... done in 25.81
Processing 4892693... done in 2.8
Processing 9038025... done in 0.65
Processing 9805439... done in 0.36
Processing 2566634... done in 0.08
Processing 2822530... done in 0.24
Processing 3135693... done in 1.23
Processing 2709108... done in 0.62
Processing 2651273... done in 56.46
Processing 2566751... done in 3.48
Processing 9569290... done in 0.36
Processing 2378030... done in 0.96
Processing 2429839... done in 1.1
Processing 6727475... done in 4.27
Processing 3486266.

In [11]:
langs = {}
for k,v in j_meta.items():
    if v['lang'] not in langs.keys():
        langs[v['lang']] = 0
    langs[v['lang']] += 1
print(langs)

{'ger': 271, '': 14, 'heb': 9, 'yid': 8, 'hun': 1, 'eng': 5, 'fre': 1, 'rus': 1}
